# Meta
In this notebook we will save out the brain, skull, and fMRI activations to then align in blender

# Setup

In [29]:
from notebook_viewer_functions import *
from functions import *
import numpy as np
import ants
import matplotlib.pyplot as plt
import os

proj_root = parent_directory()
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"
template_folder =  os.path.join(proj_root, "templates/")
output_folder = os.path.join(proj_root, "output/blender_alignment")

bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_template = ants.image_read(mni_anat_filepath)
mni_mask = ants.image_read(mni_mask_filepath)

In [ ]:
mask = generate_brain_mask(t1_image, mni_template, mni_mask)

In [ ]:
mask_vol = create_normalized_volume(mask.numpy())

In [6]:
t1_vol = create_normalized_volume(t1_image.numpy())

**Obvs still need to do method of subtraction on the vdb**

In [5]:
fmri_vol = create_normalized_volume(bold_image.numpy())

In [58]:
normalize = (lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr)))

In [73]:
def subtract_average_from_frame(bold_vol: np.ndarray):

    average = np.mean(bold_vol, axis=3)
    result = np.empty_like(bold_vol)
    for frame in range(bold_vol.shape[3]):
        result[:,:,:,frame] = bold_vol[:,:,:,frame] - average
    return result

In [74]:
def subtract_previous_frame(bold_vol: np.ndarray):
    result = np.empty_like(bold_vol)
    for frame in range(bold_vol.shape[3]):
        if frame == range(fmri_vol.shape[3])[0]:
            result[:,:,:,frame] = np.zeros_like(bold_vol[:,:,:,frame])
        else:
            result[:,:,:,frame] = bold_vol[:,:,:,frame] - bold_vol[:,:,:,frame-1]
    return result


In [83]:
def subtract_control_frame(bold_vol: np.ndarray, control_frame_idx: int):
    result = np.empty_like(bold_vol)
    for frame in range(bold_vol.shape[3]):
        result[:,:,:,frame] = bold_vol[:,:,:,frame] - bold_vol[:,:,:,control_frame_idx]
    return result

In [103]:
#threshold = (lambda arr, thresh=0.1: np.where(arr <thresh, 0.0, arr))
threshold = (lambda arr : np.where(arr < 0.3, 0.0, arr))

In [109]:
diffed_bold = threshold(normalize(subtract_control_frame(fmri_vol, 1)))

In [110]:
explore_3D_vol(diffed_bold[30,:,:,:], dim='z')

interactive(children=(IntSlider(value=92, description='slice', max=184), Output()), _dom_classes=('widget-inte…

In [19]:
with open(f'{output_folder}/mask.npy', 'wb') as f:
    np.save(f, np.array(mask_vol))

In [20]:
with open(f'{output_folder}/anat.npy', 'wb') as f:
    np.save(f, np.array(t1_vol))

In [14]:
with open(f'{output_folder}/bold.npy', 'wb') as f:
    np.save(f, np.array(diffed_bold))